# Perform Queries in our Graph

In [2]:
from rdflib import Graph
g = Graph()
g.parse("../IFC Conversion/house.ttl")

g

<Graph identifier=Nc66e8ee36bed4b3d90e332969e567bd3 (<class 'rdflib.graph.Graph'>)>

## Using SPARQL

### Art. 22§2

The doors with width below 1050mm

In [3]:
ourQuery = """
PREFIX bot: <https://w3id.org/bot#>
PREFIX beo: <https://w3id.org/beo#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT ?s ?w
WHERE{ ?s rdf:type beo:Door .
    ?s ex:width ?w .
    FILTER (?w < 1050)
}
"""

qres = g.query(ourQuery)

print( "Found these Doors with Width below 1050 mm: " )
for row in qres:
    print(f"{row.s} - door width = {row.w}")

Found these Doors with Width below 1050 mm: 
https://example.org#M_Single-Flush_0915x2032mm_398822 - door width = 915.0
https://example.org#M_Single-Flush_0915x2032mm_398909 - door width = 915.0
https://example.org#M_Single-Flush_0915x2032mm_399431 - door width = 915.0
https://example.org#M_Single-Flush_0915x2032mm_399501 - door width = 915.0
https://example.org#M_Single-Flush_0915x2032mm_399643 - door width = 915.0
https://example.org#M_Single-Flush_0915x2032mm_399925 - door width = 915.0
https://example.org#M_Single-Flush_0915x2032mm_400081 - door width = 915.0
https://example.org#M_Single-Flush_0915x2134mm_358185 - door width = 915.0
https://example.org#M_Single-Flush_0915x2134mm_358480 - door width = 915.0
https://example.org#M_Single-Flush_0915x2134mm_359949 - door width = 915.0
https://example.org#M_Single-Flush_0915x2134mm_377662 - door width = 915.0
https://example.org#M_Single-Flush_0915x2134mm_378185 - door width = 915.0
https://example.org#M_Single-Flush_0915x2134mm_378243 -

### Art. 20§2

Stairs can have at most 17 risers.

In [4]:
ourQuery = """
# Check Riser Height
PREFIX ex: <https://example.org#>
PREFIX beo: <https://w3id.org/beo#>
select * where {
    ?s ?p beo:Stair .
    ?s ex:NumberOfRailings ?rl .
    ?s ex:NumberOfRiser ?ri .
    ?s ex:RiserHeight ?rh .
    ?s ex:TreadLength ?tl
    FILTER(?ri > 17)
} limit 100
"""

qres = g.query(ourQuery)

if len(qres) > 0:
    print( "Found these Stair(s) with more than 17 risers:" )
    for row in qres:
        print(f"{row.s} : Risers = {row.ri}")
else:
    print( "No stairs have more than 17 risers." )

Found these Stair(s) with more than 17 risers:
https://example.org#PrecastStair_Stair_383689 : Risers = 18


### Art. 20§3

The height of risers may be at most 18cm.

In [5]:
ourQuery = """
# Check Riser Height
PREFIX ex: <https://example.org#>
PREFIX beo: <https://w3id.org/beo#>
select * where {
    ?s ?p beo:Stair .
    ?s ex:NumberOfRailings ?r .
    ?s ex:RiserHeight ?rh .
    ?s ex:TreadLength ?tl
    FILTER(?rh > 180)
} limit 100
"""

qres = g.query(ourQuery)

if len(qres) > 0:
    print( "Found these Stairs too high risers:" )
    for row in qres:
        print(f"{row.s} : RH = {row.rh} TL = {row.tl}")
else:
    print( "All stairs have a small enough riser height." )

All stairs have a small enough riser height.


The length of treads should be at least 23cm.

In [6]:
ourQuery = """
# Check Riser Height
PREFIX ex: <https://example.org#>
PREFIX beo: <https://w3id.org/beo#>
select * where {
    ?s ?p beo:Stair .
    ?s ex:NumberOfRailings ?r .
    ?s ex:RiserHeight ?rh .
    ?s ex:TreadLength ?tl
    FILTER(?tl < 230)
} limit 100
"""

qres = g.query(ourQuery)

if len(qres) > 0:
    print( "Found these Stairs with too short tread lengths:" )
    for row in qres:
        print(f"{row.s} : RH = {row.rh} TL = {row.tl}")
else:
    print( "All stairs have a sufficient tread length." )

All stairs have a sufficient tread length.


### Art. 20§3
The stairs and the calculated stair formula: 2xRiser + 1x TreadLength

In [7]:
ourQuery = """
# Check Stair formula
PREFIX ex: <https://example.org#>
PREFIX beo: <https://w3id.org/beo#>
select * where {
    ?s ?p beo:Stair .
    ?s ex:NumberOfRailings ?r .
    ?s ex:RiserHeight ?rh .
    ?s ex:TreadLength ?tl
    BIND( (2 * ?rh + ?tl) AS ?StairFormula )
    FILTER((?StairFormula < 590) || (?StairFormula > 630))
} limit 100
"""

qres = g.query(ourQuery)

if len(qres) > 0:
    print( "Found these Stairs with non-compliant Stair Formula: " )
    for row in qres:
        print(f"{row.s} : 2xRH+TL = {row.StairFormula}")
else:
    print("All stairs comply with the Stair Formula")

Found these Stairs with non-compliant Stair Formula: 
https://example.org#PrecastStair_Stair_397996 : 2xRH+TL = 454
